In [ ]:
from IPython.display import display, clear_output
import utils
import ipywidgets as ipw
import widgets
import datetime
import json
import os

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

SAMPLE_PREPARATION_SAMPLE_TYPES = []
for sample_preparation in CONFIG["sample_preparation"]:
    object_type = DATA_MODEL["classes"][sample_preparation]["annotations"]["openbis_label"].replace(" ", "_").upper()
    SAMPLE_PREPARATION_SAMPLE_TYPES.append(object_type)

SLABS_TYPES = []
for slab in CONFIG["slabs"]:
    object_type = DATA_MODEL["classes"][slab]["annotations"]["openbis_label"].replace(" ", "_").upper()
    SLABS_TYPES.append(object_type)
    
SAMPLES_COLLECTION_OPENBIS_PATH = CONFIG["samples_collection_openbis_path"]

sample_prep_selector = widgets.SamplePreparationMultipleSelectionWidget(CONFIG["sample_preparation"])

sample_prep_accordion = widgets.SamplePreparationAccordionWidget()

experiment_selector = widgets.ExperimentSelectionWidget()
experiment_selector.load_dropdown_box()

sample_selector_config = {
    "dropdown": {"width": "315px"},
    "details": {"width": "589px", "height": "250px"}
}
sample_selector = widgets.ObjectSelectionWidget("Sample", sample_selector_config)
sample_selector.load_dropdown_box()

instrument_selector = widgets.ObjectSelectionWidget("Instrument")
instrument_selector.load_dropdown_box()

sample_out_name_textbox = utils.Text(
    description = "Name", disabled = False, layout = ipw.Layout(width = '400px'), 
    placeholder = f"Write sample name here...", style = {'description_width': "110px"}
)

molecule_selector_list = widgets.MultipleSelectorWidget("molecule")
molecule_selector_list_output = ipw.Output()

add_molecule_button = utils.Button(
    description = 'Add molecule', disabled = False, button_style = '', 
    tooltip = 'Add molecule', layout = ipw.Layout(width = '150px', height = '25px')
)
remove_molecule_button = utils.Button(
    description = 'Remove molecule', disabled = False, button_style = '', 
    tooltip = 'Remove molecule', layout = ipw.Layout(width = '150px', height = '25px')
)
add_remove_molecule_buttons_hbox = ipw.HBox([add_molecule_button, remove_molecule_button])

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def add_molecule_widget(b):
    molecule_selector_list.add_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def remove_molecule_widget(b):
    molecule_selector_list.remove_selector()
    with molecule_selector_list_output:
        clear_output()
        display(molecule_selector_list)

def load_accordion_widgets(change):
    accordion_options = []
    tasks_properties_widgets = []
    
    for i, task in enumerate(sample_prep_selector.selector.value):
        
        task_widgets = []
        
        if task == "Dosing":
            chemical_selector_config = {
                "dropdown": {"width": "315px"},
                "details": {"width": "415px", "height": "250px"},
                "image": {"width": "220px", "height": "250px"}
            }
            chemical_selection_widget = widgets.ObjectSelectionWidget("Chemical", chemical_selector_config)
            chemical_selection_widget.load_dropdown_box()
            chemical_selection_widget.dropdown.observe(chemical_selection_widget.load_metadata, names = 'value')
            task_widgets.append(chemical_selection_widget)
        
        elif task == "Deposition":
            task_widgets.append(molecule_selector_list_output)
            task_widgets.append(add_remove_molecule_buttons_hbox)
            
        properties_widgets = widgets.ObjectPropertiesWidgets(task)
        properties_widgets.get_properties_widgets()
        task_widgets.append(properties_widgets)
        
        support_files = ipw.FileUpload(multiple = True)
        task_widgets.append(support_files)
        
        task_title = DATA_MODEL["classes"][task]["title"]
        sample_prep_accordion.set_title(i, task_title)
        accordion_options.append(ipw.VBox(task_widgets))
        
        properties_widgets.properties_widgets_detailed_dict["name"].observe(update_text, names = 'value')
        
        tasks_properties_widgets.append(properties_widgets)
    
    sample_prep_accordion.tasks_properties_widgets = tasks_properties_widgets
    sample_prep_accordion.children = accordion_options

def load_sample_metadata(change):
    if sample_selector.dropdown.value == -1:
        experiment_selector.dropdown.value = -1
        instrument_selector.dropdown.value = -1
        sample_selector.details_textbox.value = ''
        sample_out_name_textbox.value = ''
        return
    
    sample_object = OPENBIS_SESSION.get_object(sample_selector.dropdown.value)
    sample_details = utils.get_sample_details(OPENBIS_SESSION, sample_object, SAMPLE_PREPARATION_SAMPLE_TYPES, SLABS_TYPES)
    
    sample_metadata_string = (f"PermId: {sample_object.attrs.permId}\nMaterial:\n" +
                            "\n".join(sample_details["materials"]) + 
                            "\nProcesses:\n" + 
                            "\n".join(sample_details["sample_preparations"]))
    
    last_sample_preparation = sample_details["last_sample_preparation_object"]
    if last_sample_preparation:
        last_sample_preparation_object = OPENBIS_SESSION.get_object(last_sample_preparation)
        last_sample_experiment = utils.get_last_sample_experiment(OPENBIS_SESSION, sample_object, last_sample_preparation_object)
        
        # Notify the user in case the experiment changed
        if experiment_selector.dropdown.value != last_sample_experiment.permId and experiment_selector.dropdown.value != -1:
            dropdown_options_dict = {key: val for val, key in experiment_selector.dropdown.options}
            previous_experiment_name = dropdown_options_dict.get(experiment_selector.dropdown.value)
            new_experiment_name = dropdown_options_dict.get(last_sample_experiment.permId)
            display(utils.Javascript(data = f"alert('{f'Experiment was changed from {previous_experiment_name} to {new_experiment_name}!'}')"))

        instrument_object = utils.get_last_sample_instrument(OPENBIS_SESSION, last_sample_preparation_object)
        
        experiment_selector.dropdown.value = last_sample_experiment.permId
        instrument_selector.dropdown.value = instrument_object.permId
    
    sample_out_name = sample_object.props['name']
    for i, _ in enumerate(sample_prep_selector.selector.value):
        task_properties_widget = next((obj for obj in sample_prep_accordion.children[i].children if isinstance(obj, widgets.ObjectPropertiesWidgets)), None)
        task_name = task_properties_widget.properties_widgets_detailed_dict['name'].value
        if task_name:
            sample_out_name = f"{sample_out_name}_{task_name}"
    
    sample_selector.details_textbox.value = sample_metadata_string 
    sample_out_name_textbox.value = sample_out_name

def update_text(change):
    if sample_selector.dropdown.value != -1:
        selected_sample_name = next(label for label, val in sample_selector.dropdown.options if val == sample_selector.dropdown.value)
        for i, task in enumerate(sample_prep_selector.selector.value):
            task_properties_widget = next((obj for obj in sample_prep_accordion.children[i].children if isinstance(obj, widgets.ObjectPropertiesWidgets)), None)
            task_name = task_properties_widget.properties_widgets_detailed_dict['name'].value
            if len(task_name) > 0:
                selected_sample_name = f"{selected_sample_name}_{task_name}"
        sample_out_name_textbox.value = selected_sample_name

def get_widget_by_title(accordion_widget, title):
    # Find the index of the given title
    for index_str, title_text in accordion_widget._titles.items():
        if title_text == title:
            index = int(index_str)
            return accordion_widget.children[index]  # Return the corresponding widget
    return None  # Return None if the title isn't found

def create_sample_preparation_action(b):
    samples_names = [sample.props["name"] for sample in OPENBIS_SESSION.get_objects(type="SAMPLE")]
    
    if sample_out_name_textbox.value in samples_names:
        display(utils.Javascript(data = "alert('Sample name already exists!')"))
        return
    
    if experiment_selector.dropdown.value == -1:
        display(utils.Javascript(data = "alert('Select an experiment.')"))
        return
    
    if sample_selector.dropdown.value == -1:
        display(utils.Javascript(data = "alert('Select a sample.')"))
        return
    
    if instrument_selector.dropdown.value == -1:
        display(utils.Javascript(data = "alert('Select an instrument.')"))
        return
    
    # Check if sample is being prepared or if it is starting a new preparation based on the children.
    # If the sample has preparation tasks as parents and no measurements, the sample preparation steps 
    # are added to the existent preparation object. If the sample has no preparation tasks or if the sample has 
    # some measurements as children, a new preparation object is created.
    preparation_object = None
    new_preparation = True
    sample_object = OPENBIS_SESSION.get_object(sample_selector.dropdown.value)
    for sample_parent in sample_object.parents:
        sample_parent_object = OPENBIS_SESSION.get_object(sample_parent)
        if sample_parent_object.type in SAMPLE_PREPARATION_SAMPLE_TYPES:
            new_preparation = False
            sample_parent_parents = sample_parent_object.get_parents()
            for sample_parent_parent in sample_parent_parents:
                if sample_parent_parent.type == "PREPARATION":
                    preparation_object = sample_parent_parent
            
    if sample_object.children:
        for sample_child in sample_object.children:
            sample_child_object = OPENBIS_SESSION.get_object(sample_child)
            if sample_child_object.type in ["1D_MEASUREMENT", "2D_MEASUREMENT"]:
                new_preparation = True
    
    sample_out_name = sample_out_name_textbox.value
    if new_preparation:
        preparation_object = utils.create_openbis_object(
            OPENBIS_SESSION, type = "PREPARATION", 
            collection = experiment_selector.dropdown.value, 
            props = {"name": f"Preparation of {sample_out_name}"}
        )
    else:
        old_preparation_object_name = preparation_object.props["name"]
        preparation_object.props["name"] = f"{old_preparation_object_name} and {sample_out_name}"
        preparation_object.save()

    # Prepare sample parents based on method type
    methods_objects = []
    for i, task in enumerate(sample_prep_selector.selector.value):
        method_parents = [sample_selector.dropdown.value, instrument_selector.dropdown.value, preparation_object.permId]
        
        if task == "Dosing":
            dosing_widgets = get_widget_by_title(sample_prep_accordion, "Dosing")
            chemical_selector = dosing_widgets.children[0] # It is the first widget from Dosing
            if chemical_selector.dropdown.value == -1:
                display(utils.Javascript(data = "alert('Select a chemical.')"))
                return
            method_parents.append(chemical_selector.dropdown.value)
        
        elif task == "Deposition":
            selected_molecules = []
            for molecule_selector in molecule_selector_list.selectors:
                if molecule_selector.dropdown.value != -1:
                    selected_molecules.append(molecule_selector.dropdown.value)
                    
            if not selected_molecules:
                display(utils.Javascript(data = "alert('Select a molecule.')"))
                return
            
            method_parents.extend(selected_molecules)

        task_properties_widgets = sample_prep_accordion.tasks_properties_widgets[i]
        object_properties = utils.get_widget_values(task_properties_widgets.properties_widgets_detailed_dict)
                
        method_object = utils.create_openbis_object(
            OPENBIS_SESSION, type = DATA_MODEL["classes"][task]["annotations"]["openbis_label"].replace(" ", "_").upper(), 
            collection = experiment_selector.dropdown.value, props = object_properties, parents = method_parents
        )
        
        task_support_files_widget = next((obj for obj in sample_prep_accordion.children[i].children if isinstance(obj, ipw.FileUpload)), None)
        utils.upload_datasets(OPENBIS_SESSION, method_object, task_support_files_widget, "RAW_DATA")
        methods_objects.append(method_object)

    # Turn off sample visibility
    parent_sample = OPENBIS_SESSION.get_object(sample_selector.dropdown.value)
    parent_sample.props["exists"] = False
    parent_sample.save()

    sample_props = {"name": sample_out_name, "exists": True}
    utils.create_openbis_object(OPENBIS_SESSION, type = "SAMPLE", collection = SAMPLES_COLLECTION_OPENBIS_PATH, 
                                props = sample_props, parents = methods_objects)
    display(utils.Javascript(data = "alert('Upload successful!')"))

    # Reset widgets
    sample_selector.load_dropdown_box()
    sample_prep_selector.selector.value = ()
    sample_out_name_textbox.value = ""

def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

In [ ]:
sample_prep_selector.selector.observe(load_accordion_widgets, names = 'value')

# Sample Preparation

In [ ]:
# Widgets
display(sample_prep_selector)

display(utils.Markdown(data = "## Select experiment, sample, and instrument"))
display(experiment_selector, sample_selector, instrument_selector)

display(utils.Markdown(data = "## Sample Preparation Tasks"))
display(sample_prep_accordion)

display(utils.Markdown(data = "## Save Sample"))
display(sample_out_name_textbox)
display(save_close_buttons_hbox)
display(increase_buttons_size)

sample_selector.dropdown.observe(load_sample_metadata, names = 'value')
create_button.on_click(create_sample_preparation_action)
quit_button.on_click(close_notebook)

add_molecule_button.on_click(add_molecule_widget)
remove_molecule_button.on_click(remove_molecule_widget)